In [10]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

d = pd.read_csv(`r"/kaggle/input/obesetiy/ObesityDataSet_raw_and_data_sinthetic.csv")

In [11]:
d.head()

,Age,Gender,Height,Weight,CALC,FAVC,FCVC,NCP,SCC,SMOKE,CH2O,family_history_with_overweight,FAF,TUE,CAEC,MTRANS,NObeyesdad
0,21.0,Female,1.62,64.0,no,no,2.0,3.0,no,no,2.0,yes,0.0,1.0,Sometimes,Public_Transportation,Normal_Weight
1,21.0,Female,1.52,56.0,Sometimes,no,3.0,3.0,yes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,23.0,Male,1.80,77.0,Frequently,no,2.0,3.0,no,no,2.0,yes,2.0,1.0,Sometimes,Public_Transportation,Normal_Weight
3,27.0,Male,1.80,87.0,Frequently,no,3.0,3.0,no,no,2.0,no,2.0,0.0,Sometimes,Walking,Overweight_Level_I
4,22.0,Male,1.78,89.8,Sometimes,no,2.0,1.0,no,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


In [12]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Age                             2111 non-null   float64
 1   Gender                          2111 non-null   object 
 2   Height                          2111 non-null   float64
 3   Weight                          2111 non-null   float64
 4   CALC                            2111 non-null   object 
 5   FAVC                            2111 non-null   object 
 6   FCVC                            2111 non-null   float64
 7   NCP                             2111 non-null   float64
 8   SCC                             2111 non-null   object 
 9   SMOKE                           2111 non-null   object 
 10  CH2O                            2111 non-null   float64
 11  family_history_with_overweight  2111 non-null   object 
 12  FAF                             21

In [13]:
dd = d.select_dtypes(exclude='object')

In [15]:
d['NObeyesdad'] = d['NObeyesdad'].astype('category')
d['NObeyesdad'] = d['NObeyesdad'].cat.codes

In [18]:
dd = np.array(dd)
m,n = dd.shape

In [19]:
m

2111

In [20]:
dd = dd.T
y = d['NObeyesdad'].values
x = dd
x = x / 255

In [28]:
d['NObeyesdad'].unique()

array([1, 5, 6, 2, 0, 3, 4], dtype=int8)

In [21]:
dd.shape

(8, 2111)

In [22]:
x.shape

(8, 2111)

In [32]:
def generate():
    w1 = 0.1*np.random.randn(100,8)
    b1 = 0.1*np.random.randn(100,1)
    w2 = 0.1*np.random.randn(7,100)
    b2 = 0.1*np.random.randn(7,1)
    return w1,b1,w2,b2

def sigmoid(z):
    return 1/(1+np.exp(-z))

def forward(w1,b1,w2,b2,x):
    z1 = w1.dot(x)+b1
    a1 = sigmoid(z1)
    z2 = w2.dot(a1)+b2
    a2 = sigmoid(z2)
    return z1,a1,z2,a2

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def der(z):
    return sigmoid(z)*(1-sigmoid(z))
    
def back(z1,a1,z2,a2,w1,w2,x,y):
    one_y = one_hot(y)
    er = a2 - one_y
    #error in outer layer
    delt = er * der(z2)
    dw2 = 1 / m *delt.dot(a1.T)
    db2 = 1 / m * np.sum(er)
    #error in hidden layer
    delt_hid = w2.T.dot(delt) * der(z1)
    dw1 = 1 / m * delt_hid.dot(x.T)
    db1 = 1 / m * np.sum(delt_hid)
    
    return dw1,db1,dw2,db2

def update(w1,b1,w2,b2,dw1,db1,dw2,db2,a):
    w1 = w1 - a*dw1
    b1 = b1 - a *db1
    w2 = w2 - a*dw2
    b2 = b2 - a *db2
    return w1,b1,w2,b2
    

In [33]:
def get_predictions(a2):
    return np.argmax(a2, 0)

def get_accuracy(predictions, y):
    print(predictions, y)
    return np.sum(predictions == y) / y.size

def gradient(x,y,a,it):
    w1,b1,w2,b2 = generate()
    for i in range(it):
        z1,a1,z2,a2 = forward(w1,b1,w2,b2,x)
        dw1,db1,dw2,db2 = back(z1,a1,z2,a2,w1,w2,x,y)
        w1,b1,w2,b2 = update(w1,b1,w2,b2,dw1,db1,dw2,db2,a)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(a2)
            print(get_accuracy(predictions, y))
    return w1, b1, w2, b2

In [34]:
w1, b1, w2, b2 = gradient(x, y, 0.10,1000)

Iteration:  0
[4 4 4 ... 4 4 4] [1 1 1 ... 4 4 4]
0.15348176219801044
Iteration:  10
[4 4 4 ... 4 4 4] [1 1 1 ... 4 4 4]
0.15348176219801044
Iteration:  20
[4 4 4 ... 4 4 4] [1 1 1 ... 4 4 4]
0.15348176219801044
Iteration:  30
[4 4 4 ... 2 2 2] [1 1 1 ... 4 4 4]
0.09095215537659877
Iteration:  40
[2 2 2 ... 2 2 2] [1 1 1 ... 4 4 4]
0.1662719090478446
Iteration:  50
[2 2 2 ... 2 2 2] [1 1 1 ... 4 4 4]
0.1662719090478446
Iteration:  60
[2 2 2 ... 2 2 2] [1 1 1 ... 4 4 4]
0.1662719090478446
Iteration:  70
[2 2 2 ... 2 2 2] [1 1 1 ... 4 4 4]
0.1662719090478446
Iteration:  80
[2 2 2 ... 2 2 2] [1 1 1 ... 4 4 4]
0.1662719090478446
Iteration:  90
[2 2 2 ... 2 2 2] [1 1 1 ... 4 4 4]
0.1662719090478446
Iteration:  100
[2 2 2 ... 2 2 2] [1 1 1 ... 4 4 4]
0.1662719090478446
Iteration:  110
[2 2 2 ... 2 2 2] [1 1 1 ... 4 4 4]
0.1662719090478446
Iteration:  120
[2 2 2 ... 2 2 2] [1 1 1 ... 4 4 4]
0.1662719090478446
Iteration:  130
[2 2 2 ... 2 2 2] [1 1 1 ... 4 4 4]
0.1662719090478446
Iteration:  1

KeyboardInterrupt: 